## Implementation of the language models

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text import * 
from fastai.text.models import * 

[`text.models`](/text.models.html#text.models) module fully implements the [AWD-LSTM](https://arxiv.org/pdf/1708.02182.pdf) from Stephen Merity et al. The main idea of the article is to use a [RNN](http://www.pnas.org/content/79/8/2554) with dropout everywhere, but in an intelligent way. There is a difference with the usual dropout, which is why you’ll see a [`RNNDropout`](/text.models.html#RNNDropout) module: we zero things, as is usual in dropout, but we always zero the same thing according to the sequence dimension (which is the first dimension in pytorch). This ensures consistency when updating the hidden state through the whole sentences/articles. 

This being given, there are five different dropouts in the AWD-LSTM:

- the first one, embedding dropout, is applied when we look the ids of our tokens inside the embedding matrix (to transform them from numbers to a vector of float). We zero some lines of it, so random ids are sent to a vector of zeros instead of being sent to their embedding vector.
- the second one, input dropout, is applied to the result of the embedding with dropout. We forget random pieces of the embedding matrix (but as stated in the last paragraph, the same ones in the sequence dimension).
- the third one is the weight dropout. It’s the trickiest to implement as we randomly replace by 0s some weights of the hidden-to-hidden matrix inside the RNN: this needs to be done in a way that ensure the gradients are still computed and the initial weights still updated.
- the fourth one is the hidden dropout. It’s applied to the output of one of the layers of the RNN before it’s used as input of the next layer (again same coordinates are zeroed in the sequence dimension). This one isn’t applied to the last output, but rather…
- the fifth one is the output dropout, it’s applied to the last output of the model (and like the others, it’s applied the same way through the first dimension).

## Basic functions to get a model

In [ ]:
show_doc(get_language_model)

<h4 id="get_language_model"><code>get_language_model</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L179" class="source_link">[source]</a></h4>

> <code>get_language_model</code>(**`arch`**:`Callable`, **`vocab_sz`**:`int`, **`config`**:`dict`=***`None`***, **`drop_mult`**:`float`=***`1.0`***)

Create a language model from `arch` and its `config`, maybe `pretrained`.  

In [ ]:
show_doc(get_text_classifier)

<h4 id="get_text_classifier"><code>get_text_classifier</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L233" class="source_link">[source]</a></h4>

> <code>get_text_classifier</code>(**`arch`**:`Callable`, **`vocab_sz`**:`int`, **`n_class`**:`int`, **`bptt`**:`int`=***`70`***, **`max_len`**:`int`=***`1400`***, **`config`**:`dict`=***`None`***, **`drop_mult`**:`float`=***`1.0`***, **`lin_ftrs`**:`Collection`\[`int`\]=***`None`***, **`ps`**:`Collection`\[`float`\]=***`None`***) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Create a text classifier from `arch` and its `config`, maybe `pretrained`.  

This model uses an encoder taken from the `arch` on `config`. This encoder is fed the sequence by successive bits of size `bptt` and we only keep the last `max_seq` outputs for the pooling layers.

The decoder use a concatenation of the last outputs, a `MaxPooling` of all the outputs and an `AveragePooling` of all the outputs. It then uses a list of `BatchNorm`, `Dropout`, `Linear`, `ReLU` blocks (with no `ReLU` in the last one), using a first layer size of `3*emb_sz` then following the numbers in `n_layers`. The dropouts probabilities are read in `drops`.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer)). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer).

## Basic NLP modules

On top of the pytorch or the fastai [`layers`](/layers.html#layers), the language models use some custom layers specific to NLP.

In [ ]:
show_doc(EmbeddingDropout, title_level=3)

<h3 id="EmbeddingDropout"><code>class</code> <code>EmbeddingDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L55" class="source_link">[source]</a></h3>

> <code>EmbeddingDropout</code>(**`emb`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`embed_p`**:`float`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Apply dropout with probabily `embed_p` to an embedding layer `emb`.  

Each row of the embedding matrix has a probability `embed_p` of being replaced by zeros while the others are rescaled accordingly. 

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

tensor([[-0.7379, -1.3970, -0.4075, -0.1676,  2.0396,  3.2226,  0.7128],
        [-0.0000,  0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
        [-3.2579,  2.2972, -1.8704, -0.4090,  2.6477, -1.5015,  0.7158],
        [ 2.1455,  1.0571, -0.6086,  3.5700,  2.6271, -3.1353,  0.7277],
        [-3.7003, -1.8846,  0.2029, -0.6839,  0.2968, -2.0199,  1.3127],
        [-0.0000,  0.0000, -0.0000, -0.0000,  0.0000,  0.0000, -0.0000],
        [-0.0051,  2.7428,  3.0068,  0.6242,  1.2747,  0.9262,  0.4070],
        [ 1.9312,  3.0524, -1.2806,  1.5910, -2.1789, -0.1636, -3.4924]],
       grad_fn=<EmbeddingBackward>)

In [ ]:
show_doc(RNNDropout, title_level=3)

<h3 id="RNNDropout"><code>class</code> <code>RNNDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L12" class="source_link">[source]</a></h3>

> <code>RNNDropout</code>(**`p`**:`float`=***`0.5`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Dropout with probability `p` that is consistent on the seq_len dimension.  

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input, dp(tst_input)

(tensor([[[-2.1156,  0.9734,  0.2428,  0.9396,  0.4072, -0.8197,  0.3718],
          [ 0.4838,  1.3077, -0.8239, -0.6557,  1.3938,  0.6086, -0.2622],
          [ 0.2372, -0.1627,  0.3117, -0.4811, -1.0841, -0.5207, -0.5131]],
 
         [[-0.6924,  0.4122,  0.2517, -1.0120,  0.6808,  0.8800, -0.7463],
          [-0.9498,  0.7655,  0.7471, -0.2767,  1.2155, -0.1042, -2.1443],
          [-1.2342,  1.9187, -0.8481, -0.4115, -1.3223,  1.4266, -1.4150]],
 
         [[ 0.1539,  0.3142,  0.2158,  1.1411,  0.1316,  0.6158, -1.5078],
          [-1.0177, -0.9230,  0.9994,  0.1140,  0.7432,  0.4353,  0.0096],
          [-0.8231,  1.0086,  1.7685,  0.3304, -0.0896, -1.0513, -1.3017]]]),
 tensor([[[-3.0223,  1.3905,  0.0000,  0.0000,  0.5818, -0.0000,  0.5312],
          [ 0.6911,  1.8681, -0.0000, -0.0000,  1.9911,  0.0000, -0.3745],
          [ 0.3389, -0.2324,  0.0000, -0.0000, -1.5487, -0.0000, -0.7331]],
 
         [[-0.9892,  0.5889,  0.3596, -1.4458,  0.9725,  1.2571, -0.0000],
          [-1

In [ ]:
show_doc(WeightDropout, title_level=3)

<h3 id="WeightDropout"><code>class</code> <code>WeightDropout</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L24" class="source_link">[source]</a></h3>

> <code>WeightDropout</code>(**`module`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`weight_p`**:`float`, **`layer_names`**:`StrList`=***`['weight_hh_l0']`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

A module that warps another layer in which some weights will be replaced by 0 during training.  

Applies dropout of probability `weight_p` to the layers in `layer_names` of `module` in training mode. A copy of those weights is kept so that the dropout mask can change at every batch.

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, 'weight_hh_l0')

Parameter containing:
tensor([[-0.0702,  0.5725],
        [-0.3910,  0.6512],
        [-0.2203, -0.4315],
        [ 0.2750, -0.2917],
        [-0.4890, -0.3094],
        [ 0.4638, -0.3807],
        [-0.2290, -0.6964],
        [ 0.1224,  0.4043]], requires_grad=True)

It's at the beginning of a forward pass that the dropout is applied to the weights.

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, 'weight_hh_l0')

tensor([[-0.0000,  0.0000],
        [-0.6517,  0.0000],
        [-0.0000, -0.7191],
        [ 0.4583, -0.0000],
        [-0.0000, -0.0000],
        [ 0.7730, -0.6345],
        [-0.0000, -1.1607],
        [ 0.2040,  0.6739]], grad_fn=<MulBackward0>)

In [ ]:
show_doc(PositionalEncoding, title_level=3)

<h3 id="PositionalEncoding"><code>class</code> <code>PositionalEncoding</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L11" class="source_link">[source]</a></h3>

> <code>PositionalEncoding</code>(**`d`**:`int`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Encode the position with a sinusoid.  

In [ ]:
show_doc(DecoderLayer, title_level=3)

<h3 id="DecoderLayer"><code>class</code> <code>DecoderLayer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L142" class="source_link">[source]</a></h3>

> <code>DecoderLayer</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadAttention'`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Basic block of a Transformer model.  

In [ ]:
show_doc(MultiHeadAttention, title_level=3)

<h3 id="MultiHeadAttention"><code>class</code> <code>MultiHeadAttention</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L35" class="source_link">[source]</a></h3>

> <code>MultiHeadAttention</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`=***`None`***, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

MutiHeadAttention.  

In [ ]:
show_doc(MultiHeadRelativeAttention, title_level=3)

<h3 id="MultiHeadRelativeAttention"><code>class</code> <code>MultiHeadRelativeAttention</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L93" class="source_link">[source]</a></h3>

> <code>MultiHeadRelativeAttention</code>(**`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***) :: [`MultiHeadAttention`](/text.models.transformer.html#MultiHeadAttention)

MutiHeadAttention with relative positional encoding.  

In [ ]:
show_doc(SequentialRNN, title_level=3)

<h3 id="SequentialRNN"><code>class</code> <code>SequentialRNN</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L153" class="source_link">[source]</a></h3>

> <code>SequentialRNN</code>(**\*`args`**) :: [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

A sequential module that passes the reset call to its children.  

In [ ]:
show_doc(SequentialRNN.reset)

<h4 id="SequentialRNN.reset"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L155" class="source_link">[source]</a></h4>

> <code>reset</code>()

Call the `reset` function of [`self.children`](/torch_core.html#children) (if they have one).

In [ ]:
show_doc(dropout_mask)

<h4 id="dropout_mask"><code>dropout_mask</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L8" class="source_link">[source]</a></h4>

> <code>dropout_mask</code>(**`x`**:`Tensor`, **`sz`**:`Collection`\[`int`\], **`p`**:`float`)

Return a dropout mask of the same type as `x`, size `sz`, with probability `p` to cancel an element.  

In [ ]:
tst_input = torch.randn(3,3,7)
dropout_mask(tst_input, (3,7), 0.3)

tensor([[0.0000, 1.4286, 1.4286, 0.0000, 1.4286, 1.4286, 0.0000],
        [1.4286, 1.4286, 1.4286, 0.0000, 1.4286, 0.0000, 0.0000],
        [1.4286, 0.0000, 1.4286, 0.0000, 0.0000, 0.0000, 1.4286]])

Such a mask is then expanded in the sequence length dimension and multiplied by the input to do an [`RNNDropout`](/text.models.html#RNNDropout).

## Language model modules

In [ ]:
show_doc(AWD_LSTM, title_level=3)

<h3 id="AWD_LSTM"><code>class</code> <code>AWD_LSTM</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L74" class="source_link">[source]</a></h3>

> <code>AWD_LSTM</code>(**`vocab_sz`**:`int`, **`emb_sz`**:`int`, **`n_hid`**:`int`, **`n_layers`**:`int`, **`pad_token`**:`int`, **`bidir`**:`bool`=***`False`***, **`hidden_p`**:`float`=***`0.2`***, **`input_p`**:`float`=***`0.6`***, **`embed_p`**:`float`=***`0.1`***, **`weight_p`**:`float`=***`0.5`***, **`qrnn`**:`bool`=***`False`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

AWD-LSTM/QRNN inspired by https://arxiv.org/abs/1708.02182.  

In [ ]:
show_doc(Transformer, title_level=3)

<h3 id="Transformer"><code>class</code> <code>Transformer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L154" class="source_link">[source]</a></h3>

> <code>Transformer</code>(**`vocab_sz`**:`int`, **`ctx_len`**:`int`, **`n_layers`**:`int`, **`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`embed_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`True`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadAttention'`***, **`learned_pos_enc`**:`bool`=***`True`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Transformer model: https://arxiv.org/abs/1706.03762.  

In [ ]:
show_doc(TransformerXL, title_level=3)

<h3 id="TransformerXL"><code>class</code> <code>TransformerXL</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/transformer.py#L179" class="source_link">[source]</a></h3>

> <code>TransformerXL</code>(**`vocab_sz`**:`int`, **`ctx_len`**:`int`, **`n_layers`**:`int`, **`n_heads`**:`int`, **`d_model`**:`int`, **`d_head`**:`int`, **`d_inner`**:`int`, **`resid_p`**:`float`=***`0.0`***, **`attn_p`**:`float`=***`0.0`***, **`ff_p`**:`float`=***`0.0`***, **`embed_p`**:`float`=***`0.0`***, **`bias`**:`bool`=***`False`***, **`scale`**:`bool`=***`True`***, **`act`**:[`Activation`](/text.models.transformer.html#Activation)=***`<Activation.ReLU: 1>`***, **`double_drop`**:`bool`=***`True`***, **`attn_cls`**:`Callable`=***`'MultiHeadRelativeAttention'`***, **`learned_pos_enc`**:`bool`=***`False`***, **`mem_len`**:`int`=***`0`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

TransformerXL model: https://arxiv.org/abs/1901.02860.  

This is the encoder of the model with an embedding layer of `vocab_sz` by `emb_sz`, a hidden size of `n_hid`, `n_layers` layers. `pad_token` is passed to the `Embedding`, if `bidir` is True, the model is bidirectional. If `qrnn` is True, we use QRNN cells instead of LSTMs. Dropouts are `embed_p`, `input_p`, `weight_p` and `hidden_p`.

In [ ]:
show_doc(AWD_LSTM.reset)

<h4 id="AWD_LSTM.reset"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L129" class="source_link">[source]</a></h4>

> <code>reset</code>()

Reset the hidden states.  

In [ ]:
show_doc(LinearDecoder, title_level=3)

<h3 id="LinearDecoder"><code>class</code> <code>LinearDecoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L135" class="source_link">[source]</a></h3>

> <code>LinearDecoder</code>(**`n_out`**:`int`, **`n_hid`**:`int`, **`output_p`**:`float`, **`tie_encoder`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=***`None`***, **`bias`**:`bool`=***`True`***) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

To go on top of a RNNCore module and create a Language Model.  

Create a the decoder to go on top of an [`RNNCore`](/text.models.awd_lstm.html#RNNCore) encoder and create a language model. `n_hid` is the dimension of the last hidden state of the encoder, `n_out` the size of the output. Dropout of `output_p` is applied. If a `tie_encoder` is passed, it will be used for the weights of the linear layer, that will have `bias` or not.

## Classifier modules

In [ ]:
show_doc(PoolingLinearClassifier, title_level=3)

<h3 id="PoolingLinearClassifier"><code>class</code> <code>PoolingLinearClassifier</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L159" class="source_link">[source]</a></h3>

> <code>PoolingLinearClassifier</code>(**`layers`**:`Collection`\[`int`\], **`drops`**:`Collection`\[`float`\]) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Create a linear classifier with pooling.  

The last output, `MaxPooling` of all the outputs and `AvgPooling` of all the outputs are concatenated, then blocks of [`bn_drop_lin`](/layers.html#bn_drop_lin) are stacked, according to the values in [`layers`](/layers.html#layers) and `drops`.

In [ ]:
show_doc(PoolingLinearClassifier.pool)

<h4 id="PoolingLinearClassifier.pool"><code>pool</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L170" class="source_link">[source]</a></h4>

> <code>pool</code>(**`x`**:`Tensor`, **`bs`**:`int`, **`is_max`**:`bool`)

Pool the tensor along the seq_len dimension.  

The input tensor `x` (of batch size `bs`) is pooled along the batch dimension. `is_max` decides if we do an `AvgPooling` or a `MaxPooling`.

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(WeightDropout.forward)

<h4 id="WeightDropout.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L42" class="source_link">[source]</a></h4>

> <code>forward</code>(**\*`args`**:`ArgStar`)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(EmbeddingDropout.forward)

<h4 id="EmbeddingDropout.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L64" class="source_link">[source]</a></h4>

> <code>forward</code>(**`words`**:`LongTensor`, **`scale`**:`Optional`\[`float`\]=***`None`***) → `Tensor`

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(RNNDropout.forward)

<h4 id="RNNDropout.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L19" class="source_link">[source]</a></h4>

> <code>forward</code>(**`x`**:`Tensor`) → `Tensor`

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(WeightDropout.reset)

<h4 id="WeightDropout.reset"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L49" class="source_link">[source]</a></h4>

> <code>reset</code>()

In [ ]:
show_doc(PoolingLinearClassifier.forward)

<h4 id="PoolingLinearClassifier.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L175" class="source_link">[source]</a></h4>

> <code>forward</code>(**`input`**:`Tuple`\[`Tensor`, `Tensor`\]) → `Tuple`\[`Tensor`, `Tensor`, `Tensor`\]

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(LinearDecoder.forward)

<h4 id="LinearDecoder.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/models/awd_lstm.py#L147" class="source_link">[source]</a></h4>

> <code>forward</code>(**`input`**:`Tuple`\[`Tensor`, `Tensor`\]) → `Tuple`\[`Tensor`, `Tensor`, `Tensor`\]

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

## New Methods - Please document or move to the undocumented section